In [1]:
import sys
sys.path.append("../")

In [2]:
from __future__ import division
from full_prod_DRA import *
from buchi import buchi_from_ltl
import numpy as np
from env_sensing_error_small import *
import scipy
# from plot_path_for_prod import *
from graphviz import Source
from qnetwork import *
from utils import *
import matplotlib.pyplot as plt
from Plot_Path import *
import tensorflow as tf
from dra_planning import *
import time
from dra_planning import convert_path_to_action

/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B"
# LTL = "[] (p1 -> !(X p1) U (p2 || p3) ) && []<>p1"
# LTL = "T && []<>A && []<>B"
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C && []!D"
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C"
# LTL = "<>(A && <>(B && <> T))"
# LTL = "<>(A && <>B) && <>[]T && []!C"
LTL = "<>(A && <>T) && []!C"
# LTL = "<>(A && <>(B && <>T)) && []<>(A||T) && []<>B && []!C"
# LTL = "<>(A && <>(B && <>T)) && []!C"
# LTL = "<>(A && <>D) && <>(B && <>E) && []<>T && []<>(D || E) && []!C"

LEARNING_RATE = 0.0015
GAMMA = 0.99
# GAMMA = 0.7
TAU = 0.001
BUFFER_SIZE = 10**6
MINIBATCH_SIZE = 64
RANDOM_SEED = 191
MAX_EPISODES = 50000
MAX_EPISODE_LEN = 500
file_appendix = "Pure_Imitation_Large_" + time.ctime()[4:16].replace("  ","").replace(" ","_").replace(":","-") + LTL
# file_appendix = "Feb8_03-47"
SUMMARY_DIR = './results/tf_ddqn_' + file_appendix
SAVE_DIR = "./saved_model/" + file_appendix + "/ddqn.ckpt"
EXPLORATION_RATE = 0.7
LR_DECAY_TRUNCATION = -800
RESTORE = 0
if sys.platform == "darwin":
    DEVICE = "/device:CPU:0"
else:
    DEVICE = "/device:GPU:0"

In [18]:
env = CurrentWorld(LTL)
prod_planner = Prod_Planning(env, LTL)
# with open("my.dot", "r") as dotfile:
#     text = dotfile.read()
# Source(text)

In [19]:
guide_policy = {}
prod_planner.get_global_opt()
prod_planner.get_opt_rabin()
for rabin_s in env.rabin.states:
    for s_x in range(env.shape[0]):
        for s_y in range(env.shape[1]):
            env.step(np.random.randint(env.nA))
            prod_planner.update_wfts_ap()
            local_plan = prod_planner.get_local_opt((s_x,s_y), prod_planner.get_next_ltl(rabin_s))
            if local_plan == None:
                guide_policy[(s_x,s_y,rabin_s)] = np.random.randint(env.nA)
            else:
                guide_policy[(s_x,s_y,rabin_s)] = convert_path_to_action(local_plan)
            
        